In [24]:
from models.transformer.Preprocess import Preprocessor
from models.transformer import transformer
import tensorflow as tf


def evaluate(sentence, output):
  # 입력 문장에 대한 전처리
  # 디코더의 예측 시작
  for i in range(preprocessor.MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)


    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    print('=====predictions=====\n', predictions)
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, preprocessor.END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루       프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)
  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)


def predict(sentence_tokenized, output_vector):
  prediction = evaluate(sentence_tokenized, output_vector)
  print('=====encoded=====\n', prediction)
  # prediction == 디코더가 리턴한 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = preprocessor.tokenizer.decode(
    [i for i in prediction if i < preprocessor.tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [26]:
if __name__ == '__main__':
  vocab_path = r'D:\banimo_diary\models\vocab.txt'
  sentence = ['으악 이거 왜 이렇게 졸라 안되는거야 짜증나 죽겠누!']
  preprocessor = Preprocessor(vocab_path, sentence)
  sentence_tokenized, output_vector = preprocessor(vocab_path, sentence)

  model = transformer.transformer(
    vocab_size=preprocessor.tokenizer.vocab_size+2,
    num_layers=2,
    dff=512,
    d_model=256,
    num_heads=8,
    dropout=.1)

  model.load_weights(r'D:\banimo_diary\models\save\weights\transformer_weight100.h5')
  output = predict(sentence_tokenized, output_vector)
  print(len(output))

reg_sentence: return(preprocessed) >  ['으악 이거 왜 이렇게 졸라 안되는거야 짜증나 죽겠누 !']
load_tokenizer: return(self.tokenizer) >  <SubwordTextEncoder vocab_size=8247>
tokenized_for_eval: return(tokenized_input, tokenized_output) >  self.tokenized_input[0]= tf.Tensor(
[[8247 8227 8147 8179 8227 8140 8124 8023 8227 8148 8171 8225 8168 8167
  8023 8227 8144 8147 8023 8227 8148 8171 8226 8151 8126 8225 8169 8131
  8023 8227 8152 8175 8226 8148 8179 8023 8227 8140 8127 8226 8135 8143
  8226 8129 8139 8225 8168 8167 8227 8140 8179 8023 8227 8158 8147 8227
  8157 8148 8226 8121 8143 8023 8227 8154 8180 8225 8169 8151 8226 8127
  8123 8024 8248]], shape=(1, 73), dtype=int32) self.tokenized_output= tf.Tensor([[8247]], shape=(1, 1), dtype=int32)
(1, 8249, 256)
(1, 8249, 256)
=====predictions=====
 tf.Tensor(
[[[-20.007692  -19.884024  -20.186596  ... -20.00737   -20.022373
    -1.7427542]]], shape=(1, 1, 8249), dtype=float32)
=====predictions=====
 tf.Tensor([[[-20.4706   -19.919289 -19.706457 ... -20.47122  -